## Model Testing

In [1]:
%matplotlib inline
import os
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import configuration as config

from utils.vocabulary import Vocabulary
from utils.caption_generator import CaptionGenerator
from model import MemeModel
from keras.models import load_model
from keras.preprocessing import image
from utils.yanan_lm import text_processing, tokenize_unigram
from utils.yanan_lm import unigram_V, unigrams_prob
from utils.yanan_lm import ngram_prob, perplexity, add_k_smoothing

Using TensorFlow backend.


## Prepare Meme Captioner

In [2]:
checkpoint_path = 'model5.16/train/'
vocab_file = 'batches/word_count.txt'
dataset_dir = 'batches/part-0-to-11960/'
model_file = 'small-conv/model.h5'

In [3]:
def build_model(dataset_dir, image_format='jpeg'):
    model = MemeModel('inference',
                      vocab_file,
                      model_file=model_file,
                      dataset_dir=dataset_dir)
    model.build(image_format)
    return model

def feed_image(sess, encoded_image):
    initial_state = sess.run(fetches="lstm/initial_state:0",
                             feed_dict={"image_feed:0": encoded_image})
    return initial_state

def inference_step(sess, input_feed, state_feed):
    softmax_output, state_output = sess.run(
        fetches=["softmax:0", "lstm/state:0"],
        feed_dict={
            "input_feed:0": input_feed,
            "lstm/state_feed:0": state_feed,
        })
    return softmax_output, state_output, None

# Creates a function that restores a model from checkpoint
def create_restore_fn(checkpoint_path, saver):
    if tf.gfile.IsDirectory(checkpoint_path):
        checkpoint_path = tf.train.latest_checkpoint(checkpoint_path)
        if not checkpoint_path:
            raise ValueError("No checkpoint file found in: %s" % checkpoint_path)

    def _restore_fn(sess):
        tf.logging.info("Loading model from checkpoint: %s", checkpoint_path)
        saver.restore(sess, checkpoint_path)
        tf.logging.info("Successfully loaded checkpoint: %s",
                        os.path.basename(checkpoint_path))
        
    return _restore_fn

# Builds the inference graph from a configuration object.
def build_graph_from_config(data_dir, checkpoint_path, image_format='jpeg'):
    tf.logging.info("Building model.")
    model = build_model(data_dir, image_format)
    saver = tf.train.Saver(tf.global_variables())
    return create_restore_fn(checkpoint_path, saver), model

In [4]:
tf.reset_default_graph()
restore_fn, mememodel = build_graph_from_config(dataset_dir,
                                                checkpoint_path,
                                                image_format='jpeg')

INFO:tensorflow:Building model.
Initializing the model's parameters...
Mapping image embeddings...
(1, 128)
Building the LSTM model...
(1, 100)
Setting up the global step tensor...


In [5]:
# Create the vocabulary.
vocab = Vocabulary(vocab_file)

INFO:tensorflow:Initializing vocabulary from file: batches/word_count.txt
INFO:tensorflow:Created vocabulary with 7412 words


In [6]:
mememodel.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 148, 148, 32)      896       
_________________________________________________________________
conv2 (Conv2D)               (None, 146, 146, 16)      4624      
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 73, 73, 16)        0         
_________________________________________________________________
droput1 (Dropout)            (None, 73, 73, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 85264)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 128)               10913920  
__________

In [7]:
sess = tf.InteractiveSession()
restore_fn(sess)

INFO:tensorflow:Loading model from checkpoint: model5.16/train/model.ckpt-2000000
INFO:tensorflow:Restoring parameters from model5.16/train/model.ckpt-2000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-2000000


In [8]:
generator = CaptionGenerator(feed_image, 
                             inference_step, 
                             vocab,
                             max_caption_length=8)

## Prepare Ngram tester

In [9]:
training_set = 'utils/brown.train.txt'
unk_threshold = 5
ADD_K_SMOOTHING = 'add_k_smoothing'
LINER_INT = 'liner interpolation'
NO_SMOOTHING = 'no smoothing'

In [10]:
# Tokenization
train_text = text_processing(training_set)
train_token = tokenize_unigram(train_text)

In [11]:
unigram_count, replaced_tokens_train = unigram_V(train_token, unk_threshold)
vocabulary = set(unigram_count.keys())

In [12]:
# generate unigram probablity dict
uni_prob_dict = {}
uni_prob_dict = unigram_count.copy()
unigrams_prob_dict = unigrams_prob(uni_prob_dict)

V = len(vocabulary)
print("Vocabulary length", V)

Vocabulary length 11331


In [13]:
# generate trigram probability dict
trigram_prob_dict = ngram_prob(3, replaced_tokens_train, unigram_count)

# generate bigram probability dict
bigram_prob_dict = ngram_prob(2, replaced_tokens_train, unigram_count)

In [14]:
# Caption pre-processing
def text_processing(text, STOP_token='_STOP_'):
    txt = text.replace('\n',' '+STOP_token+'\n')
    puncts = '!"#$%&()*+,-./:;<=>?@[\\]^`{|}~'
    for p in puncts:
        txt = txt.replace(p, ' ')
    return txt

## Caption and Testing

In [21]:
testdir = 'evaldataset/no_memes/'
testdir = [os.path.join(testdir, t) for t in os.listdir(testdir)]
k_ls = (0.0000001,0.000001,0.00001,0.0001,0.01,0.1,1)
basestring = '| <img width="50" alt="50" src="{}"> | {} / {:.2f} |  |'
for filename in testdir:
    if os.path.exists(filename):
        # Caption image
        img = image.load_img(filename, target_size=(150, 150))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        preds = mememodel.model.predict(x)
        captions = generator.beam_search(sess, preds)
        candidates = ''
        output = ''
        for i, caption in enumerate(captions):
            cap = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            cap = ' '.join(cap)
            if i == 0:
                output = cap
            candidates += cap + '\n'
        print(filename, '\n', candidates)
        dev_text = text_processing(candidates)
        perps = 0
        for k in k_ls:
            tri_addk_prob_dict = add_k_smoothing(3, replaced_tokens_train, unigram_count, k, V)
            perps += perplexity(dev_text,3,tri_addk_prob_dict,ADD_K_SMOOTHING)
        print(basestring.format(filename, output, perps / len(k_ls)), end='\n\n')

evaldataset/no_memes/COCO_test2014_000000437409.jpg 
 ask ask ask burnt seats fora
ask ask ask burnt burnt seats
ask ask ask burnt burnt fora

| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000437409.jpg"> | ask ask ask burnt seats fora / 1751.52 |  |

evaldataset/no_memes/COCO_test2014_000000581919.jpg 
 musica musica starve victoria victoria victoria
musica musica musica victoria victoria victoria
musica musica musica starve victoria victoria

| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000581919.jpg"> | musica musica starve victoria victoria victoria / 1751.52 |  |

evaldataset/no_memes/COCO_test2014_000000437984.jpg 
 facebook facebook 100000 100000 100000 matter
facebook facebook 100000 100000 100000 100000
facebook facebook 100000 100000 100000 maldita

| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000437984.jpg"> | facebook facebook 100000 100000 100000 matter / 1751.52 |  |

evaldataset/no_memes/COCO_test2

In [20]:
'{:.2f}'.format(13434.23949)

'13434.24'

| Imagen | Leyenda Modelo A / Perplejidad   | Leyenda Modelo B / Perplejidad |
|:------:|:----------------:|:-------------:|
| <img width="50" alt="50" src="evaldataset/memes/20.jpg"> | brownies brownies brownies brownies church shake / 80.88927376112007 | fuck off |
| <img width="50" alt="50" src="evaldataset/memes/12.png.jpg"> | lucky rush rush rush rush rush / 80.88927376112007 | fuck off |

| Imagen | Leyenda Modelo `A` / Perplejidad  | Leyenda Modelo `B` / Perplejidad |
|:------:|:----------------:|:-------------:|
| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000437409.jpg"> | ask ask ask burnt seats fora / 1751.52 |  |
| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000581919.jpg"> | musica musica starve victoria victoria victoria / 1751.52 |  |
| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000437984.jpg"> | facebook facebook 100000 100000 100000 matter / 1751.52 |  |
| <img width="50" alt="50" src="evaldataset/no_memes/COCO_test2014_000000581911.jpg"> | kanji kanji matter matter matter matter / 80.89 |  |
